# Module 6: The Cost-Based Optimizer (CBO)
## Goal: Shatter the illusion that the database "knows" everything.

In previous chapters, we learned about the physical reality of disks (Seek vs. Scan) and indexing (B-Trees). But who decides whether to use that B-Tree or just scan the whole disk?

Meet **The Optimizer**.

The Optimizer is the "Brain" of the database. It is not magic; it is a mathematician running an auction. Before every query, it looks at available paths (Index Scan, Seq Scan, Bitmap Scan), estimates the "Cost" of each based on stored statistics, and picks the cheapest one.

**The catch?** It doesn't look at the actual data. It looks at a *summary* of the data (Statistics). If the summary is wrong, the Optimizer hallucinates.

### The Physics of Data
1.  **Statistics (The Map):** The DB stores histograms and row counts in a separate metadata table.
2.  **Cost (The Currency):** The DB assigns a generic "cost unit" to every operation (e.g., Reading a page = 1.0, Processing a row = 0.01).
3.  **Plan Stability:** Once a plan is picked, the DB follows it blindly, even if it leads to a cliff.

---


## 1. Setup and Connection

We will use **PostgreSQL** for this module, as it has a robust and transparent Cost-Based Optimizer.

In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json

# Database Connection Parameters
DB_PARAMS = {
    "host": "db_int_opt",
    "port": 5432,
    "user": "admin",
    "password": "password",
    "dbname": "db_int_opt"
}

def get_db_connection():
    conn = psycopg2.connect(**DB_PARAMS)
    conn.autocommit = True
    return conn

print("✅ Connected to Postgres.")

## 2. Experiment 6.1: The Statistics (Histograms)
How does the database know that `PREMIUM` users are rare (5%) but `ACTIVE` users are common (70%) without counting them?

It uses Histograms. We will load your specific `users` data and see if the database can predict the skew.

#### Step 1: Prepare the Data
We load `users.csv` into Postgres and explicitly run `ANALYZE` to build the statistics.

In [ ]:
iconn = get_db_connection()
cur = conn.cursor()

# 1. Create table matching your file schema
cur.execute("DROP TABLE IF EXISTS users_stats;")
cur.execute("""
    CREATE TABLE users_stats (
        user_id INTEGER,
        name TEXT,
        email TEXT,
        account_status VARCHAR(20),
        signup_date TIMESTAMP,
        is_email_verified BOOLEAN
    );
""")

# 2. Load data
print("⏳ Loading users.csv into Postgres...")
df = pd.read_csv('../data/users.csv')

# Efficient Bulk Insert
buffer = StringIO()
df.to_csv(buffer, index=False, header=False)
buffer.seek(0)
cur.copy_expert("COPY users_stats FROM STDIN WITH CSV", buffer)

# 3. CRITICAL STEP: Force the Database to study the data
cur.execute("ANALYZE users_stats;")
print("✅ Data Loaded and Analyzed.")

## Step 2: The Prediction Test
We will ask the database to **explain** two queries. We won't run them; we just want the Plan Rows (The Estimate).
1. `SELECT * FROM users_stats WHERE account_status = 'ACTIVE'`
2. `SELECT * FROM users_stats WHERE account_status = 'PREMIUM'`

**Hypothesis**: If the DB is smart, it should predict a higher number for 'ACTIVE' than 'PREMIUM', even though the query looks identical.

In [ ]:
def get_estimate(status_value):
    query = f"EXPLAIN (FORMAT JSON) SELECT * FROM users_stats WHERE account_status = '{status_value}'"
    cur.execute(query)
    plan = cur.fetchone()[0][0]['Plan']
    return plan['Plan Rows']

# Get estimates from the Optimizer
est_active = get_estimate('ACTIVE')
est_premium = get_estimate('PREMIUM')

# Get actuals (Ground Truth) from the Disk
cur.execute("SELECT count(*) FROM users_stats WHERE account_status = 'ACTIVE'")
real_active = cur.fetchone()[0]

cur.execute("SELECT count(*) FROM users_stats WHERE account_status = 'PREMIUM'")
real_premium = cur.fetchone()[0]

print(f"ACTIVE  (Majority) -> Estimated: {est_active}, Actual: {real_active}")
print(f"PREMIUM (Minority) -> Estimated: {est_premium},   Actual: {real_premium}")

### Step 3: Visualization

In [ ]:
labels = ['ACTIVE', 'PREMIUM']
estimates = [est_active, est_premium]
actuals = [real_active, real_premium]

x = range(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(8, 5))
bar1 = ax.bar([i - width/2 for i in x], estimates, width, label='Optimizer Estimate', color='skyblue')
bar2 = ax.bar([i + width/2 for i in x], actuals, width, label='Actual Rows', color='salmon')

ax.set_ylabel('Row Count')
ax.set_title('Experiment 6.1: Can the DB see the Skew?')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()

#### Step 4: Analysis
Look at how closely the Blue Bar (what the database thought it had) matches the Red Bar (what it actually had).
1. **The Skew is Visible**: The database correctly knows that ACTIVE users are the vast majority (~70k) and PREMIUM users are a tiny minority (~5k).
2. **The Accuracy**:
    - **Active**: Estimated 70,097 vs Actual 70,111 (99.9% accuracy).
    - **Premium**: Estimated 4,967 vs Actual 4,871 (98% accuracy).

**What does this prove?**
It proves that the Cost-Based Optimizer (CBO) is not guessing blindly. When you ran the `ANALYZE` command in the setup, the database built a statistical "Map" of your data.

**The Lesson**: The database didn't actually count the rows to give you that blue bar. It looked at its "cheat sheet" (Histogram) and said, "Based on my last survey, I expect to find about 70,097 rows."

---

## 3. Experiment 6.2: The Auction (Comparing Paths)
The CBO uses "Cost" to decide between strategies.
- **Sequential Scan**: Good for `ACTIVE` users (reading 70% of the disk).
- **Index Scan**: Good for `PREMIUM` users (reading 5% of the disk).

We will force the database to use the wrong method for `ACTIVE` users and measure the penalty.

##### Step 1: Setup Indices
We'll use the same `users_stats` table but add an index on `account_status`.

In [ ]:
# 1. Create Index
print("⏳ Building Index on account_status...")
cur.execute("CREATE INDEX idx_status ON users_stats(account_status);")
cur.execute("ANALYZE users_stats;")
print("✅ Index Built.")

#### Step 2: The "Tipping Point"
We query for `ACTIVE` users.
- **Hypothesis**: Reading 70k rows via Index (Random I/O) is slower than reading the whole file (Seq I/O).

In [ ]:
query = "SELECT count(*) FROM users_stats WHERE account_status = 'ACTIVE'"

# 1. Run with CBO's Natural Choice (Should be Seq Scan)
cur.execute("SET enable_seqscan = ON;")
cur.execute("SET enable_indexscan = ON;") 
start_nat = time.time()
cur.execute(f"EXPLAIN (ANALYZE, FORMAT JSON) {query}")
result_nat = cur.fetchone()[0][0]
end_nat = time.time()

cost_nat = result_nat['Plan']['Total Cost']
time_nat = (end_nat - start_nat) * 1000 # ms
algo_nat = result_nat['Plan']['Node Type']

# 2. Force the Bad Choice (Index Scan for 70% of data)
cur.execute("SET enable_seqscan = OFF;") # DISABLE Seq Scan
start_bad = time.time()
cur.execute(f"EXPLAIN (ANALYZE, FORMAT JSON) {query}")
result_bad = cur.fetchone()[0][0]
end_bad = time.time()

cost_bad = result_bad['Plan']['Total Cost']
time_bad = (end_bad - start_bad) * 1000 # ms
algo_bad = result_bad['Plan']['Node Type']

# Reset
cur.execute("SET enable_seqscan = ON;")

print(f"Natural ({algo_nat}): Cost={cost_nat:.2f}, Time={time_nat:.2f}ms")
print(f"Forced  ({algo_bad}): Cost={cost_bad:.2f}, Time={time_bad:.2f}ms")

#### Step 3: Visualization

In [ ]:
methods = [f"Natural\n({algo_nat})", f"Forced\n({algo_bad})"]
times = [time_nat, time_bad]
costs = [cost_nat, cost_bad]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Chart 1: Execution Time (Reality)
sns.barplot(x=methods, y=times, ax=ax1, palette=["green", "red"])
ax1.set_title("Actual Execution Time (ms)")
ax1.set_ylabel("Milliseconds")

# Chart 2: Optimizer Cost (Prediction)
sns.barplot(x=methods, y=costs, ax=ax2, palette=["green", "red"])
ax2.set_title("Optimizer 'Cost' Units (Prediction)")
ax2.set_ylabel("Cost Units")

plt.tight_layout()
plt.show()

#### Step 4: Analysis

**Why did the Optimizer fail?**
You have encountered The RAM Illusion.

The Optimizer's cost model is conservative; it assumes the data is cold and sitting on a slow disk. It penalizes "Random I/O" (Index Scans) heavily because moving a mechanical disk head takes ~5-10ms.

**However**: Your dataset (100k rows) is tiny.
- The entire table fits into the Docker container's RAM (Memory).
- "Seeking" in RAM takes nanoseconds, not milliseconds.
- Because the penalty for seeking vanished, the overhead of the Index Scan disappeared, and it beat the Sequential Scan.

#### The "Senior Architect" Lesson
If this table were 100 GB instead of 10 MB:
- The Right Chart (Prediction) would look exactly the same.
- The Left Chart (Reality) would flip. The Red bar (Index Scan) would be 100x taller than the Green bar, because the disk would be thrashing back and forth trying to read 70% of the data randomly.

**Conclusion for this Lab**: You have proven that Performance is context-dependent. The Optimizer made the "safe" choice for a scalable system, even if it wasn't the absolute fastest choice for this tiny, cached dataset.

----

## 4. Experiment 6.3: When the Optimizer Lies (Stale Stats)
The Optimizer relies on `pg_stats`. If we delete data but don't tell the database, the map is wrong.

#### Step 1: Create Stale Scenario
We will create a fresh table, analyze it, then delete 90% of the data (all the `ACTIVE` users).

In [ ]:
# 1. Setup Table
cur.execute("DROP TABLE IF EXISTS stale_demo;")
cur.execute("CREATE TABLE stale_demo AS SELECT * FROM users_stats;")
cur.execute("ANALYZE stale_demo;") # Fresh stats

# 2. Get Initial Estimate
cur.execute("EXPLAIN (FORMAT JSON) SELECT * FROM stale_demo;")
initial_est = cur.fetchone()[0][0]['Plan']['Plan Rows']

# 3. The Purge (Delete all ACTIVE users)
# NOTE: We are NOT running ANALYZE after this.
cur.execute("DELETE FROM stale_demo WHERE account_status = 'ACTIVE';") 

# 4. Get Stale Estimate
cur.execute("EXPLAIN (FORMAT JSON) SELECT * FROM stale_demo;")
stale_est = cur.fetchone()[0][0]['Plan']['Plan Rows']

# 5. Get Real Count
cur.execute("SELECT count(*) FROM stale_demo;")
real_count = cur.fetchone()[0]

print(f"Initial Estimate: {initial_est}")
print(f"Stale Estimate:   {stale_est} (The Lie)")
print(f"Actual Rows:      {real_count}")

#### Step 2: Visualization

In [ ]:
labels = ['Before Delete (Est)', 'After Delete (Est)', 'After Delete (Actual)']
values = [initial_est, stale_est, real_count]

plt.figure(figsize=(8, 5))
bars = plt.bar(labels, values, color=['gray', 'red', 'green'])
plt.title('Experiment 6.3: The Stale Statistics Problem')
plt.ylabel('Row Count')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom')

plt.show()

### Step 3: Analysis
**The Hallucination**: The database still thinks it has 100k rows (Red Bar), even though you deleted 70k of them (Green Bar).

**Why this matters**: If you run a Join now, the database will reserve memory for 100k rows. Since only ~30k exist, that memory is wasted. Conversely, if you added 1M rows but the DB thought you had 100, it would reserve too little memory, forcing the join to spill to disk (crashing performance).